# Hardcover API Exploration

## 1. Import Libraries

In [3]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from pydantic import SecretStr, HttpUrl, ValidationError, Field
from pydantic_settings import BaseSettings, SettingsConfigDict
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from gql.transport.exceptions import TransportQueryError
import requests

## 2. Define Tests

In [4]:
# --- Determine Project Root ---
# Navigate up from the current working directory until we find '.env' or 'pyproject.toml'
def find_project_root(marker_files=(".env", "pyproject.toml")):
    """Finds the project root directory by searching upwards for marker files."""
    current_path = Path.cwd() # Directory where Jupyter kernel was started
    print(f"Notebook Kernel CWD: {current_path}")
    for directory in [current_path] + list(current_path.parents):
        for marker in marker_files:
            if (directory / marker).exists():
                print(f"Project root found at: {directory} (marker: {marker})")
                return directory
    raise FileNotFoundError(f"Could not find project root. Looked for {marker_files} starting from {current_path}")

try:
    PROJECT_ROOT = find_project_root()
    dotenv_path = PROJECT_ROOT / '.env'
except FileNotFoundError as e:
     print(f"❌ ERROR: {e}")
     print("Make sure your .env file exists in the main project directory (where pyproject.toml is).")
     raise SystemExit("Stopping execution.")


# Define settings model specifically for this notebook
class ExplorationSettings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file=dotenv_path, # Use the calculated path to .env in the root
        env_file_encoding='utf-8',
        extra='ignore'
    )
    # Use validation_alias to match the exact variable names in the .env file
    hardcover_api_key: SecretStr | None = Field(None, validation_alias='HARDCOVER_API_KEY')
    hardcover_api_url: HttpUrl | None = Field(None, validation_alias='HARDCOVER_API_URL')


# Load environment variables from .env file explicitly FIRST
# This makes them available for pydantic-settings to read
if dotenv_path.exists():
    # override=True ensures .env vars take precedence over existing env vars
    load_dotenv(dotenv_path=dotenv_path, override=True)
    print(f".env file found and loaded from: {dotenv_path.resolve()}")
else:
    # This case should ideally be caught by find_project_root, but added as safety
    print(f"⚠️ WARNING: .env file not found at {dotenv_path.resolve()}. Cannot load settings from it.")

# Load settings using pydantic-settings (will read from loaded env vars)
try:
    settings = ExplorationSettings()
    print("Settings loaded.")
    # Check if the values were actually loaded
    if not settings.hardcover_api_key or not settings.hardcover_api_key.get_secret_value():
         print("⚠️ WARNING: HARDCOVER_API_KEY not found, empty, or failed to load from .env file!")
         API_KEY = None # Ensure API_KEY is None if not loaded
    else:
         API_KEY = settings.hardcover_api_key.get_secret_value()
         # Never print the full key in logs or outputs!
         print(f"Hardcover API Key: Loaded (starts with '{API_KEY[:5]}...')")

    if not settings.hardcover_api_url:
         print("⚠️ WARNING: HARDCOVER_API_URL not found, invalid, or failed to load from .env file!")
         API_URL = None # Ensure API_URL is None if not loaded
    else:
        API_URL = str(settings.hardcover_api_url)
        print(f"Hardcover API URL: {API_URL}")

except ValidationError as e:
    print(f"❌ ERROR: Failed to load or validate settings: {e}")
    print("Check if HARDCOVER_API_KEY and HARDCOVER_API_URL are correctly formatted in your .env file.")
    raise SystemExit("Stopping due to configuration errors.")


# --- Proceed only if essential settings are loaded ---
if not API_KEY:
    raise SystemExit("CRITICAL: API Key (HARDCOVER_API_KEY) could not be loaded. Check .env file and previous warnings.")
if not API_URL:
    raise SystemExit("CRITICAL: API URL (HARDCOVER_API_URL) could not be loaded. Check .env file and previous warnings.")

print("\nConfiguration loaded successfully. Ready for Cell 2.")

Notebook Kernel CWD: d:\Documents\phantom\phantom-enrichment\notebooks\api_explorations
Project root found at: d:\Documents\phantom\phantom-enrichment (marker: .env)
.env file found and loaded from: D:\Documents\phantom\phantom-enrichment\.env
Settings loaded.
Hardcover API Key: Loaded (starts with 'Beare...')
Hardcover API URL: https://api.hardcover.app/v1/graphql

Configuration loaded successfully. Ready for Cell 2.


In [7]:
# Configure the transport layer
# The Hardcover docs are slightly ambiguous. Let's try 'Bearer <token>' first as it's standard.
# If this fails with auth errors, we might try just the token itself.
auth_header = API_KEY
# Alternative if Bearer fails: auth_header = API_KEY

transport = RequestsHTTPTransport(
    url=API_URL,
    headers={'Authorization': auth_header},
    verify=True, # Enable SSL verification
    retries=3,   # Retry up to 3 times on transient network errors
)

# Create the GraphQL client
client = Client(transport=transport, fetch_schema_from_transport=False) # Set to True if you want to try fetching schema

print("GraphQL client configured.")
print(f"Using Authorization Header: {auth_header[:15]}...") # Show prefix only
print(f"Target URL: {API_URL}")

GraphQL client configured.
Using Authorization Header: Bearer eyJhbGci...
Target URL: https://api.hardcover.app/v1/graphql


## 3. Basic Test

In [8]:
# The basic test query from Hardcover docs
test_query = gql("""
    query TestMe {
      me {
        id
        username
      }
    }
""")

print("Attempting 'me' query to test authentication...")

try:
    response = client.execute(test_query)
    print("\n✅ Success! Response:")
    print(json.dumps(response, indent=2))
    print("\nAuthentication seems successful. You can proceed to search.")

except TransportQueryError as e:
    print(f"\n❌ GraphQL Error during 'me' query: {e}")
    if e.errors:
        print("GraphQL Errors:")
        for error in e.errors:
            print(f"- {error}")
    if e.response:
        print(f"HTTP Status Code: {e.response.status_code}")
        print(f"HTTP Response Body: {e.response.text}")
    print("\nCheck:")
    print("1. Is HARDCOVER_API_URL correct in your .env file?")
    print("2. Is HARDCOVER_API_KEY correct (the Bearer token)?")
    print("3. Try changing the auth_header format in Cell 2 (Bearer vs just token)?")


except requests.exceptions.RequestException as e:
    print(f"\n❌ Network/HTTP Error during 'me' query: {e}")
    print("Check network connection and the HARDCOVER_API_URL.")

except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")

Attempting 'me' query to test authentication...

✅ Success! Response:
{
  "me": [
    {
      "id": 32959,
      "username": "desesseintes"
    }
  ]
}

Authentication seems successful. You can proceed to search.


## 4. Search Test

In [11]:
# --- Parameters for your search ---
search_title = "Austerlitz"
search_author = "Sebald" # Keep it simple for now

# --- Attempt 3: Try direct arguments ---
search_query_string = gql("""
  query SearchBooksWithDirectArgs($titleArg: String!, $authorArg: String) {
    # Still assuming 'books', trying direct args 'title' and 'authorName' (guess)
    books(title: $titleArg, authorName: $authorArg, limit: 5) { # Added limit
      # --- Requesting simple fields ---
      id
      slug
      title
      authors {
          id
          name
      }
      yearPublished
      # --- End of simple fields ---
    }
  }
""")
# Variables matching the argument names defined in the query above
variables = {"titleArg": search_title, "authorArg": search_author}


print(f"Attempting book search with Title='{search_title}', Author='{search_author}' using direct arguments...")
# print("Using Query:\n", search_query_string) # Uncomment to see query
# print("Variables:", variables) # Uncomment to see variables

try:
    response = client.execute(search_query_string, variable_values=variables)

    print("\n✅ Success! Response:")
    print(json.dumps(response, indent=2))
    print("\n--- Analysis ---")
    print("1. Did direct arguments work?")
    print("2. If yes, note the structure and fields.")
    print("3. If not, the error should tell us which argument ('title' or 'authorName') is wrong.")

except TransportQueryError as e:
    print(f"\n❌ GraphQL Error during search query: {e}")
    if hasattr(e, 'errors') and e.errors:
        print("GraphQL Server Errors:")
        for error in e.errors:
             print(f"- {error}")
             if isinstance(error, dict):
                 if 'message' in error:
                     print(f"  Hint: {error['message']}")
                     # Look specifically for unknown argument errors
                     if "Unknown argument" in error['message']:
                         arg_name = error['message'].split('"')[1] # Extract argument name
                         print(f"  >> The argument '{arg_name}' is incorrect. Need to find the right name.")
                     elif "has no argument named" in error['message']:
                          print(f"  >> The argument mentioned is incorrect.")
    else:
         print("Could not extract detailed GraphQL errors.")

    print("\n--- What to try next ---")
    print("1. If an argument name was wrong (e.g., 'authorName'), try alternatives like 'author'.")
    print("2. **Seriously consider using the Hardcover GraphQL console now.** Guessing argument names can take a long time. The console's 'Docs' or 'Schema' panel will list the exact arguments for the 'books' query.")
    print("   - URL: (Check Hardcover docs/settings page)")
    print("   - Header: 'Authorization' : 'YOUR_TOKEN_STRING'")
    print("   - Look for the 'books' query and click it to see its details/arguments.")


except requests.exceptions.RequestException as e:
    print(f"\n❌ Network/HTTP Error during search query: {e}")
    print("Check network connection and the HARDCOVER_API_URL.")

except Exception as e:
    print(f"\n❌ An unexpected error occurred: {e}")

Attempting book search with Title='Austerlitz', Author='Sebald' using direct arguments...

❌ GraphQL Error during search query: {'message': "'books' has no argument named 'authorName'", 'extensions': {'path': '$.selectionSet.books', 'code': 'validation-failed'}}
GraphQL Server Errors:
- {'message': "'books' has no argument named 'authorName'", 'extensions': {'path': '$.selectionSet.books', 'code': 'validation-failed'}}
  Hint: 'books' has no argument named 'authorName'
  >> The argument mentioned is incorrect.

--- What to try next ---
1. If an argument name was wrong (e.g., 'authorName'), try alternatives like 'author'.
2. **Seriously consider using the Hardcover GraphQL console now.** Guessing argument names can take a long time. The console's 'Docs' or 'Schema' panel will list the exact arguments for the 'books' query.
   - URL: (Check Hardcover docs/settings page)
   - Header: 'Authorization' : 'YOUR_TOKEN_STRING'
   - Look for the 'books' query and click it to see its details/argu

This is getting us nowhere. We'll use introspection.

## 5. Introspection

In [12]:
from gql import gql
import json

# The introspection query focused on query fields and their arguments
introspection_query = gql("""
    query IntrospectionQueryForQueries {
      __schema {
        queryType {
          name
          fields {
            name
            description
            args {
              name
              description
              type { ...TypeRef }
              defaultValue
            }
            type { ...TypeRef }
          }
        }
      }
    }

    fragment TypeRef on __Type {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
          }
        }
      }
    }
""")

print("Attempting introspection query to discover schema details...")

try:
    # No variables needed for this query
    response = client.execute(introspection_query)

    print("\n✅ Introspection Query Successful! Schema details received.")
    # The response can be large, print it formatted
    # print(json.dumps(response, indent=2)) # Uncomment to see the full raw response

    # --- Let's process the response to find the 'books' query ---
    query_type = response.get('__schema', {}).get('queryType', {})
    fields = query_type.get('fields', [])

    found_books_query = False
    for field in fields:
        if field.get('name') == 'books':
            found_books_query = True
            print("\n--- Found 'books' Query Field ---")
            print(f"Description: {field.get('description')}")
            print("Arguments:")
            if field.get('args'):
                for arg in field['args']:
                    # Dig into the type structure to get the base type name and wrappers
                    type_info = arg.get('type', {})
                    type_str = ""
                    if type_info.get('kind') == 'NON_NULL':
                        type_str += f"{type_info.get('ofType', {}).get('name')}!"
                    elif type_info.get('kind') == 'LIST':
                         # Simplified representation for lists
                         nested_type = type_info.get('ofType', {})
                         nested_name = nested_type.get('name')
                         if nested_type.get('kind') == 'NON_NULL':
                              nested_name = f"{nested_type.get('ofType',{}).get('name')}!"
                         type_str += f"[{nested_name}]"
                    else:
                        type_str = type_info.get('name')

                    print(f"  - Name: {arg.get('name')}")
                    print(f"    Type: {type_str}")
                    print(f"    Description: {arg.get('description')}")
                    print(f"    Default Value: {arg.get('defaultValue')}")
            else:
                print("  (No arguments)")
            break # Stop after finding 'books'

    if not found_books_query:
        print("\n⚠️ Could not find a query field named 'books' in the introspection results.")
        print("Available query fields found:")
        for field in fields:
            print(f"- {field.get('name')}")
        print("Maybe the search query has a different name (e.g., 'search', 'findBooks')?")


except Exception as e:
    print(f"\n❌ An error occurred during the introspection query: {e}")
    # Check if it's a specific GraphQL error indicating introspection is disabled
    if "introspection" in str(e).lower():
         print("Introspection might be disabled on this server.")
    else:
         # Handle potential transport errors etc. as before
         if isinstance(e, TransportQueryError) and hasattr(e, 'errors') and e.errors:
             print("GraphQL Server Errors:")
             for error in e.errors: print(f"- {error}")

Attempting introspection query to discover schema details...

✅ Introspection Query Successful! Schema details received.

--- Found 'books' Query Field ---
Description: An array relationship
Arguments:
  - Name: distinct_on
    Type: [books_select_column!]
    Description: distinct select on columns
    Default Value: None
  - Name: limit
    Type: Int
    Description: limit the number of rows returned
    Default Value: None
  - Name: offset
    Type: Int
    Description: skip the first n rows. Use only with order_by
    Default Value: None
  - Name: order_by
    Type: [books_order_by!]
    Description: sort the rows by one or more columns
    Default Value: None
  - Name: where
    Type: books_bool_exp
    Description: filter the rows returned
    Default Value: None


## 6. Introspecting `books_bool_exp`

In [13]:
from gql import gql
import json

# Introspection query to find details about the 'books_bool_exp' input type
introspect_input_type_query = gql("""
    query IntrospectInputObjectType($typeName: String!) {
      __type(name: $typeName) {
        kind
        name
        description
        inputFields { # Use inputFields for INPUT_OBJECT types
          name
          description
          type { ...TypeRef }
          defaultValue
        }
      }
    }

    fragment TypeRef on __Type {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
          }
        }
      }
    }
""")

input_type_name = "books_bool_exp" # The type name we found for the 'where' argument
print(f"Attempting introspection query for Input Type: {input_type_name}...")

try:
    response = client.execute(introspect_input_type_query, variable_values={"typeName": input_type_name})

    print(f"\n✅ Introspection for '{input_type_name}' Successful!")
    # print(json.dumps(response, indent=2)) # Uncomment for full response

    type_details = response.get('__type', {})
    if type_details and type_details.get('kind') == 'INPUT_OBJECT':
        print(f"\n--- Details for '{type_details.get('name')}' (Input Object) ---")
        print(f"Description: {type_details.get('description')}")
        print("Input Fields (available for filtering):")
        if type_details.get('inputFields'):
            for field in type_details['inputFields']:
                # Simplified type representation
                type_info = field.get('type', {})
                type_str = type_info.get('name', 'UnknownType')
                if type_info.get('kind') == 'NON_NULL':
                    type_str = f"{type_info.get('ofType', {}).get('name')}!"
                elif type_info.get('kind') == 'LIST':
                     nested_type = type_info.get('ofType', {})
                     nested_name = nested_type.get('name')
                     if nested_type.get('kind') == 'NON_NULL':
                          nested_name = f"{nested_type.get('ofType',{}).get('name')}!"
                     type_str = f"[{nested_name}]"

                print(f"  - Name: {field.get('name')}")
                print(f"    Type: {type_str}")
                print(f"    Description: {field.get('description')}")

                # --- Specifically look for operators like _eq ---
                if type_str.endswith('_comparison_exp'): # Fields ending like this often hold operators
                   print(f"    >> This field likely contains comparison operators (_eq, _gt, etc.). Need to inspect '{type_str}' type further.")
                elif field.get('name') in ['title', 'author', 'authors', 'name']: # Potential direct match fields?
                   print(f"    >> Potential candidate for direct matching.")
                elif field.get('name') == '_and' or field.get('name') == '_or' or field.get('name') == '_not':
                   print(f"    >> Logical operator for combining conditions.")

        else:
            print("  (No input fields found)")
    elif type_details:
        print(f"\n⚠️ Found type '{input_type_name}', but it's a {type_details.get('kind')}, not an INPUT_OBJECT.")
    else:
        print(f"\n❌ Could not find details for type '{input_type_name}'.")


except Exception as e:
    print(f"\n❌ An error occurred during the input type introspection query: {e}")

Attempting introspection query for Input Type: books_bool_exp...

✅ Introspection for 'books_bool_exp' Successful!

--- Details for 'books_bool_exp' (Input Object) ---
Description: Boolean expression to filter rows from the table "books". All fields are combined with a logical 'AND'.
Input Fields (available for filtering):
  - Name: _and
    Type: [books_bool_exp!]
    Description: None
    >> Logical operator for combining conditions.
  - Name: _not
    Type: books_bool_exp
    Description: None
    >> Logical operator for combining conditions.
  - Name: _or
    Type: [books_bool_exp!]
    Description: None
    >> Logical operator for combining conditions.
  - Name: activities_count
    Type: Int_comparison_exp
    Description: None
    >> This field likely contains comparison operators (_eq, _gt, etc.). Need to inspect 'Int_comparison_exp' type further.
  - Name: alternative_titles
    Type: json_comparison_exp
    Description: None
    >> This field likely contains comparison operat

## 7. Searching Using Exact Match

In [16]:
from gql import gql
import json

# --- Parameters ---
search_title_exact = "Austerlitz"

# --- Attempt 6: Remove limit from description field ---
search_query_string = gql("""
  query SearchBooksExactTitle($titleExact: String!) {
    books(
        limit: 5,
        where: {
            title: { _eq: $titleExact }
        }
    ) {
      id
      slug
      title
      description # REMOVED (limit: 500) from here
      yearPublished: release_year
      authors: contributions {
          role
          contributor {
              id
              name
          }
      }
      series: featured_book_series {
          bookNumber: number_in_series # Still guessing this field name
          series {
              id
              name
          }
      }
    }
  }
""")
variables = {"titleExact": search_title_exact}

print(f"Attempting book search with EXACT title='{search_title_exact}' (removed limit from description)...")
# print("Using Query:\n", search_query_string)
# print("Variables:", variables)

# --- Keep the same try/except block ---
try:
    response = client.execute(search_query_string, variable_values=variables)
    print("\n✅ Success! Response:")
    print(json.dumps(response, indent=2))
    print("\n--- Analysis ---")
    print("1. Did removing the limit on description fix the error?")
    print("2. If successful, examine the returned 'authors' and 'series' structures closely.")
    print("3. We now have a working query for exact title match!")

except TransportQueryError as e:
     print(f"\n❌ GraphQL Error during search query: {e}")
     if hasattr(e, 'errors') and e.errors:
         print("GraphQL Server Errors:")
         for error in e.errors:
             print(f"- {error}")
             if isinstance(error, dict):
                 if 'message' in error:
                     print(f"  Hint: {error['message']}")
                     # Check for issues with relationship fields
                     if "contributions" in error['message'] or "contributor" in error['message']:
                         print("  >> Error likely in the 'authors: contributions {...}' part. Check nested field names.")
                     if "featured_book_series" in error['message'] or "number_in_series" in error['message']:
                         print("  >> Error likely in the 'series: featured_book_series {...}' part. Check nested field names.")
     print("\n--- What to try next ---")
     print("1. If it still fails, simplify the query further: remove the 'authors' and 'series' sections entirely and just fetch `id`, `title`, `description`, `yearPublished` to ensure the basic `where` clause is working.")
     print("2. Use the Hardcover GraphQL console to verify the field names within `contributions` and `featured_book_series`.")


except Exception as e:
     print(f"\n❌ An unexpected error occurred: {e}")

Attempting book search with EXACT title='Austerlitz' (removed limit from description)...

❌ GraphQL Error during search query: {'message': "field 'role' not found in type: 'contributions'", 'extensions': {'path': '$.selectionSet.books.selectionSet.contributions.selectionSet.role', 'code': 'validation-failed'}}
GraphQL Server Errors:
- {'message': "field 'role' not found in type: 'contributions'", 'extensions': {'path': '$.selectionSet.books.selectionSet.contributions.selectionSet.role', 'code': 'validation-failed'}}
  Hint: field 'role' not found in type: 'contributions'
  >> Error likely in the 'authors: contributions {...}' part. Check nested field names.

--- What to try next ---
1. If it still fails, simplify the query further: remove the 'authors' and 'series' sections entirely and just fetch `id`, `title`, `description`, `yearPublished` to ensure the basic `where` clause is working.
2. Use the Hardcover GraphQL console to verify the field names within `contributions` and `feature